In [125]:
"""
Fishy:
    -Improve feature encoding to have proper ordering instead of random numbers
    which currently influence classification accuracy:
    https://datascience.stackexchange.com/questions/72343/encoding-with-ordinalencoder-how-to-give-levels-as-user-input
   
   -It's possible for train-test split to split data in such a way, that
   after encoding, X_train and X_test have different numbers of features.
   Split has to be rerun to fix it. First encode, then split again?
   BUT IT STILL HAS TO BE ENCODED AND SPLIT BEFORE STARTING CROSS VALIDATION
   AND SEQUENTIAL FEATURE SELECTION. MAYBE APPEND DURING SPLIT AND THEN SPLIT
   AGAIN?
"""

Pick up: make sure all datasets pass feature encoding

SyntaxError: invalid syntax (Temp/ipykernel_644/621060733.py, line 15)

ERROR:root:Cannot parse: 15:5: Pick up: make sure all datasets pass feature encoding
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\lab_black.py", line 218, in format_cell
    formatted_code = _format_code(cell)
  File "C:\ProgramData\Anaconda3\lib\site-packages\lab_black.py", line 29, in _format_code
    return format_str(src_contents=code, mode=FileMode())
  File "C:\ProgramData\Anaconda3\lib\site-packages\black.py", line 725, in format_str
    src_node = lib2to3_parse(src_contents.lstrip(), mode.target_versions)
  File "C:\ProgramData\Anaconda3\lib\site-packages\black.py", line 836, in lib2to3_parse
    raise exc from None
black.InvalidInput: Cannot parse: 15:5: Pick up: make sure all datasets pass feature encoding


In [104]:
## SKLEARN
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import CategoricalNB, MultinomialNB, GaussianNB
from sklearn import metrics
from sklearn.compose import make_column_transformer

import numpy as np

np.set_printoptions(suppress=True)
import pandas as pd
from platform import python_version

import scipy

import random

# works, sort of only. Possible additional commas that shouldn't be there.
%load_ext nb_black

print("Libs imported. Python version is: ", python_version())

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black
Libs imported. Python version is:  3.9.7


<IPython.core.display.Javascript object>

In [90]:
# utility functions

cols_mushroom = [
    "labels",
    "cap-shape",
    "cap-surface",
    "cap-color",
    "bruises",
    "odor",
    "gill-attachment",
    "gill-spacing",
    "gill-size",
    "gill-color",
    "stalk-shape",
    "stalk-root",
    "stalk-surface-above-ring",
    "stalk-surface-below-ring",
    "stalk-color-above-ring",
    "stalk-color-below-ring",
    "veil-type",
    "veil-color",
    "ring-number",
    "ring-type",
    "spore-print-color",
    "population",
    "habitat",
]

cols_car = ["buying", "maintenance", "doors", "passengers", "boot", "safety", "labels"]

cols_audiology = [
    "age_gt_60",
    "air",
    "airBoneGap",
    "ar_c",
    "ar_u",
    "bone",
    "boneAbnormal",
    "bser",
    "history_buzzing",
    "history_dizziness",
    "history_fluctuating",
    "history_fullness",
    "history_heredity",
    "history_nausea",
    "history_noise",
    "history_recruitment",
    "history_ringing",
    "history_roaring",
    "history_vomiting",
    "late_wave_poor",
    "m_at_2k",
    "m_cond_lt_1k",
    "m_gt_1k",
    "m_m_gt_2k",
    "m_m_sn",
    "m_m_sn_gt_1k",
    "m_m_sn_gt_2k",
    "m_m_sn_gt_500",
    "m_p_sn_gt_2k",
    "m_s_gt_500",
    "m_s_sn",
    "m_s_sn_gt_1k",
    "m_s_sn_gt_2k",
    "m_s_sn_gt_3k",
    "m_s_sn_gt_4k",
    "m_sn_2_3k",
    "m_sn_gt_1k",
    "m_sn_gt_2k",
    "m_sn_gt_3k",
    "m_sn_gt_4k",
    "m_sn_gt_500",
    "m_sn_gt_6k",
    "m_sn_lt_1k",
    "m_sn_lt_2k",
    "m_sn_lt_3k",
    "middle_wave_poor",
    "mod_gt_4k",
    "mod_mixed",
    "vmod_s_mixed",
    "mod_s_sn_gt_500",
    "mod_sn",
    "mod_sn_gt_1k",
    "mod_sn_gt_2k",
    "mod_sn_gt_3k",
    "mod_sn_gt_4k",
    "mod_sn_gt_500",
    "notch_4k",
    "notch_at_4k",
    "o_ar_c",
    "o_ar_u",
    "s_sn_gt_1k",
    "s_sn_gt_2k",
    "s_sn_gt_4k",
    "speech",
    "static_normal",
    "tymp",
    "viith_nerve_signs",
    "wave_V_delayed",
    "waveform_ItoV_prolonged",
    "p-index",
    "labels",
]

cols_wine = [
    "labels",
    "alcohol",
    "mallic-acid",
    "alcalinity",
    "ash",
    "magnesium",
    "total-phenols",
    "flavonids",
    "nonflavonid-phenols",
    "proanthocyanins",
    "color-intensity",
    "hue",
    "od-of-diluted-wines",
    "proline",
]

"""
https://archive.ics.uci.edu/ml/datasets/car+evaluation
0-5 -> data
6 -> labels
"""


def load_car():
    df_car = pd.read_csv(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data",
        header=None,
        names=cols_car,
    )
    # mappings using indexes:
    # X = df_car.loc[:, :5].values
    # y = df_car.loc[:, 6].values
    labels_col = df_car.pop("labels")
    df_car.insert(0, "labels", labels_col)
    # replace 5more in doors to 5
    # df_car.loc[df_car['doors'] == '5more', 'doors'] = '5'
    # df_car["doors"] = pd.to_numeric(df_car["doors"])
    # replace more in passengers to 5
    return df_car


"""
https://archive.ics.uci.edu/ml/datasets/mushroom
1-22 -> data
0 -> labels
"""


def load_mushroom():
    df_mushroom = pd.read_csv(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data",
        header=None,
        names=cols_mushroom,
    )
    # index mappings
    # X = df_mushroom.loc[:, 1:].values
    # y = df_mushroom.loc[:, 0].values
    return df_mushroom


"""
https://archive.ics.uci.edu/ml/datasets/Audiology+%28Standardized%29
0:length-2 -> data
length-1 unique id (p1-p200)
length -> labels
"""


def load_audiology():
    df_audiology = pd.read_csv(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/audiology/audiology.standardized.data",
        header=None,
        names=cols_audiology,
    )
    # index mapping
    # length = len(df_audiology.columns)
    # X = df_audiology.loc[:, : length - 3].values
    # y = df_audiology.loc[:, length - 1].values
    df_audiology = df_audiology.drop("p-index", axis=1)
    labels_col = df_audiology.pop("labels")
    df_audiology.insert(0, "labels", labels_col)
    return df_audiology


"""
https://www.alldatascience.com/classification/wine-dataset-analysis-with-python/
1:length -> data
0 -> labels
"""


def load_wine():
    df_wine = pd.read_csv(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data",
        header=None,
        names=cols_wine,
    )
    # index mappings
    # length = len(df_wine.columns)
    # X = df_wine.loc[:, 1:].values
    # y = df_wine.loc[:, 0].values
    return df_wine

<IPython.core.display.Javascript object>

In [118]:
# Choose dataset
# dataset = load_car()
# dataset = load_mushroom()
dataset = load_audiology()
# dataset = load_wine()

print(dataset.info())
print("First five records:")
print(dataset.head())

# Extract to X and y
X_cat = dataset.loc[:, dataset.columns != "labels"]
y_cat = dataset.loc[:, "labels"]

print("Size of X: ", np.shape(X_cat))
print("Size of y: ", np.shape(y_cat))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 70 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   labels                   200 non-null    object
 1   age_gt_60                200 non-null    object
 2   air                      200 non-null    object
 3   airBoneGap               200 non-null    object
 4   ar_c                     200 non-null    object
 5   ar_u                     200 non-null    object
 6   bone                     200 non-null    object
 7   boneAbnormal             200 non-null    object
 8   bser                     200 non-null    object
 9   history_buzzing          200 non-null    object
 10  history_dizziness        200 non-null    object
 11  history_fluctuating      200 non-null    object
 12  history_fullness         200 non-null    object
 13  history_heredity         200 non-null    object
 14  history_nausea           200 non-null    o

<IPython.core.display.Javascript object>

In [119]:
max_seed_val = 2 ** 32 - 1

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(
    X_cat, y_cat, test_size=0.25, random_state=random.randrange(0, max_seed_val)
)
print(
    "Data has been split. Train dataset contains ",
    np.shape(X_train),
    ", test dataset contains ",
    np.shape(X_test),
)

Data has been split. Train dataset contains  (150, 69) , test dataset contains  (50, 69)


<IPython.core.display.Javascript object>

In [120]:
# Collectors of values

cols_one_hot = [
    "cap-shape",
    "cap-surface",
    "cap-color",
    "bruises",
    "odor",
    "gill-attachment",
    "gill-spacing",
    "gill-size",
    "gill-color",
    "stalk-shape",
    "stalk-root",
    "stalk-surface-above-ring",
    "stalk-surface-below-ring",
    "stalk-color-above-ring",
    "stalk-color-below-ring",
    "veil-type",
    "veil-color",
    "ring-number",
    "ring-type",
    "spore-print-color",
    "habitat",
    "age_gt_60",
    "airBoneGap",
    "boneAbnormal",
    "bser",
    "history_buzzing",
    "history_dizziness",
    "history_fluctuating",
    "history_fullness",
    "history_heredity",
    "history_nausea",
    "history_noise",
    "history_recruitment",
    "history_ringing",
    "history_roaring",
    "history_vomiting",
    "late_wave_poor",
    "m_at_2k",
    "m_cond_lt_1k",
    "m_gt_1k",
    "m_m_gt_2k",
    "m_m_sn",
    "m_m_sn_gt_1k",
    "m_m_sn_gt_2k",
    "m_m_sn_gt_500",
    "m_p_sn_gt_2k",
    "m_s_gt_500",
    "m_s_sn",
    "m_s_sn_gt_1k",
    "m_s_sn_gt_2k",
    "m_s_sn_gt_3k",
    "m_s_sn_gt_4k",
    "m_sn_2_3k",
    "m_sn_gt_1k",
    "m_sn_gt_2k",
    "m_sn_gt_3k",
    "m_sn_gt_4k",
    "m_sn_gt_500",
    "m_sn_gt_6k",
    "m_sn_lt_1k",
    "m_sn_lt_2k",
    "m_sn_lt_3k",
    "middle_wave_poor",
    "mod_gt_4k",
    "mod_mixed",
    "vmod_s_mixed",
    "mod_s_sn_gt_500",
    "mod_sn",
    "mod_sn_gt_1k",
    "mod_sn_gt_2k",
    "mod_sn_gt_3k",
    "mod_sn_gt_4k",
    "mod_sn_gt_500",
    "notch_4k",
    "notch_at_4k",
    "s_sn_gt_1k",
    "s_sn_gt_2k",
    "s_sn_gt_4k",
    "static_normal",
    "tymp",
    "viith_nerve_signs",
    "wave_V_delayed",
    "waveform_ItoV_prolonged",
]

cols_ordinal = [
    "buying",
    "maintenance",
    "doors",
    "passengers",
    "boot",
    "safety",
    "population",
    "air",
    "ar_c",
    "ar_u",
    "bone",
    "o_ar_c",
    "o_ar_u",
    "speech",
]


def collect_current_one_hot_columns(argNames):
    # make list of all values and create steps for them
    return list(set(cols_one_hot).intersection(argNames))


def collect_current_ordinal_columns(argNames):
    # make list of all values and create steps for them
    return list(set(cols_ordinal).intersection(argNames))

<IPython.core.display.Javascript object>

In [122]:
# Encode the columns using one hot and ordinal encoders
ohe = OneHotEncoder()
oe = OrdinalEncoder()

# Make column transformer

column_list = X_train.select_dtypes(include="object").columns
columns_one_hot = collect_current_one_hot_columns(column_list)
columns_ordinal = collect_current_ordinal_columns(column_list)

column_transform = make_column_transformer(
    (ohe, collect_current_one_hot_columns(column_list)),
    (oe, collect_current_ordinal_columns(column_list)),
)

# Apply transformation
encoded_X_train = column_transform.fit_transform(X_train)
encoded_X_test = column_transform.fit_transform(X_test)

if scipy.sparse.issparse(encoded_X_train):
    encoded_X_train = encoded_X_train.toarray()

if scipy.sparse.issparse(encoded_X_test):
    encoded_X_test = encoded_X_test.toarray()

print(type(encoded_X_train))
print(type(encoded_X_test))
print(np.shape(encoded_X_train))
print(np.shape(encoded_X_test))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(150, 126)
(50, 104)


<IPython.core.display.Javascript object>

In [116]:
# Create a Bayes Classifier
gnb = CategoricalNB()

# Train the model using the training sets
gnb.fit(encoded_X_train, y_train)

# Predict the response for test dataset
y_pred = gnb.predict(encoded_X_test)

<IPython.core.display.Javascript object>

In [117]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy:", metrics.accuracy_score(y_test, y_pred) * 100, "%")

Accuracy: 93.89463318562285 %


<IPython.core.display.Javascript object>

In [ ]:
"""
useful links:

https://dev.to/codinghappinessweb/analysing-dataset-using-naive-bayes-classifier-3d7o
https://towardsdatascience.com/15-tips-and-tricks-for-jupyter-notebook-that-will-ease-your-coding-experience-e469207ac95c
https://towardsdatascience.com/guide-to-encoding-categorical-features-using-scikit-learn-for-machine-learning-5048997a5c79
https://towardsdatascience.com/naive-bayes-classifier-how-to-successfully-use-it-in-python-ecf76a995069
https://scikit-learn.org/stable/modules/naive_bayes.html#multinomial-naive-bayes
"""